In [1]:
!pip install duckdb==1.4.4 --upgrade --quiet
!pip install jupysql duckdb-engine --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.9/197.9 kB 14.4 MB/s eta 0:00:00


In [9]:
# ############################################################################################
# BLOKK 1: STRUKTUUR JA INITSIALISEERIMINE
# Seadistab SQL magic laienduse ja loob dbt stiilis (lowercase keywords) tabelid.
# Initsialiseerib andmebaasi TAI API juurkirjega ja seadistab kuvamise piirangud.
# ############################################################################################

import os, duckdb, sqlalchemy, requests, shutil, time
from sqlalchemy import create_engine, text
from google.colab import drive
from datetime import datetime

# 1. Drive ja Ühendus
drive.mount('/content/drive')
db_folder = '/content/drive/MyDrive/Colab_TAI_Andmed'
db_path = os.path.join(db_folder, 'tai_statistika.db')
os.makedirs(db_folder, exist_ok=True)

engine = create_engine(f'duckdb:///{db_path}')

# 2. SQL Magic seadistamine
%load_ext sql
%sql engine
%config SqlMagic.displaylimit = 20

with engine.begin() as conn:
    # 3. Laiendused
    conn.execute(text("install httpfs from core; load httpfs;"))
    conn.execute(text("install json from core; load json;"))

    # 4. Tabelite struktuur dbt stiilis (lowercase keywords)
    conn.execute(text("""
        create table if not exists tai_meta (
            id              varchar primary key,
            src_url         varchar,
            parent_id       varchar default '',
            obj_type        varchar default 'l',
            ee_description  varchar default '',
            table_name      varchar default '',
            db_schema       varchar default '',
            dt_updated_api  timestamp default '1900-01-01',
            dt_created      timestamp default current_timestamp,
            dt_checked      timestamp default '1900-01-01'
        );

        create table if not exists tai_load_sessions (
            session_id      varchar primary key,
            dt_start        timestamp,
            dt_end          timestamp,
            status          varchar,
            notes           varchar default ''
        );

        create table if not exists tai_load_log (
            table_id        varchar,
            session_id      varchar,
            dt_start        timestamp,
            dt_end          timestamp,
            dt_updated_api  timestamp,
            result          varchar,
            notes           varchar default ''
        );
    """))

    # 5. INIT: Juurkirje lisamine
    root_exists = conn.execute(text("select count(*) from tai_meta")).scalar()
    if root_exists == 0:
        conn.execute(text("""
            insert into tai_meta (
                id,
                src_url,
                parent_id,
                obj_type,
                ee_description
            ) values (
                'Andmebaas',
                'https://statistika.tai.ee/api/v1/et/Andmebaas',
                '',
                'l',
                'Juurkataloog'
            )
        """))

print(f"✅ BLOKK 1 Valmis. Andmebaas: {db_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
✅ BLOKK 1 Valmis. Andmebaas: /content/drive/MyDrive/Colab_TAI_Andmed/tai_statistika.db


In [3]:
%%sql

checkpoint;

Running query in 'duckdb:////content/drive/MyDrive/Colab_TAI_Andmed/tai_statistika.db'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Success


In [10]:
# ############################################################################################
# BLOKK 2: CRAWLER JA METAANDMETE SÜNKRONISEERIMINE
# Käib läbi TAI API kataloogipuu, uuendab metaandmed ja märgib kontrolli aja (dt_checked).
# ############################################################################################

import time
from datetime import datetime

sessiooni_algus = datetime.now()
session_id = sessiooni_algus.strftime('%Y%m%d%H%M%S')
final_status = 'COMPLETED'

with engine.begin() as trans_s:
    trans_s.execute(text("""
        insert into tai_load_sessions (session_id, dt_start, status, notes)
        values (:sid, :start, 'RUNNING', 'Crawler start')
    """), {"sid": session_id, "start": sessiooni_algus})

with engine.begin() as trans_q:
    trans_q.execute(text("""
        create or replace table queue (id varchar primary key);
        insert into queue (id) values ('Andmebaas');
    """))

print(f"🚀 BLOKK 2: Korje algas. Sessioon: {session_id}")

try:
    while True:
        with engine.connect() as conn:
            row = conn.execute(text("""
                select
                    m.id,
                    m.src_url,
                    m.db_schema
                from tai_meta m
                join queue q on m.id = q.id
                limit 1
            """)).fetchone()

        if not row:
            break

        p_id, p_url, p_schema = row

        try:
            time.sleep(0.3)

            with engine.begin() as trans:
                trans.execute(text(f"""
                    create or replace table temp_json as
                    select * from read_json('{p_url}', format='array')
                """))

                cols = [r[0] for r in trans.execute(text("describe temp_json")).fetchall()]
                upd_col_expr = ("try_cast(updated as timestamp)" if "updated" in cols
                               else "cast('1900-01-01' as timestamp)")

                trans.execute(text(f"""
                    insert into tai_meta (
                        id, src_url, parent_id, obj_type, ee_description,
                        table_name, db_schema, dt_updated_api, dt_checked
                    )
                    select
                        case when :p_id = 'Andmebaas' then id else :p_id || '.' || id end,
                        :p_url || '/' || id,
                        :p_id,
                        type,
                        text,
                        case
                            when type = 't' then 'tai_' || regexp_replace(split_part(:p_id, '.', -1), '[^a-zA-Z0-9_]', '_', 'g') || '_' || regexp_replace(id, '[^a-zA-Z0-9_]', '_', 'g') || '_px'
                            else ''
                        end,
                        case
                            when :p_id = 'Andmebaas' then 'tai_' || regexp_replace(id, '[^a-zA-Z0-9_]', '_', 'g')
                            else :p_schema
                        end,
                        {upd_col_expr},
                        :sess_algus
                    from temp_json
                    on conflict (id) do update set
                        dt_updated_api = excluded.dt_updated_api,
                        dt_checked = excluded.dt_checked
                """), {"p_id": p_id, "p_url": p_url, "p_schema": p_schema, "sess_algus": sessiooni_algus})

                trans.execute(text("""
                    insert into queue (id)
                    select case when :p_id = 'Andmebaas' then id else :p_id || '.' || id end
                    from temp_json
                    where type = 'l'
                    on conflict do nothing
                """), {"p_id": p_id})

                trans.execute(text("delete from queue where id = :id"), {"id": p_id})

            print(f"✅ {p_id}")

        except KeyboardInterrupt:
            final_status = 'INTERRUPTED'
            raise
        except Exception as e:
            with engine.begin() as trans_err:
                trans_err.execute(text("""
                    insert into tai_load_log (table_id, session_id, dt_start, dt_end, result, notes)
                    values (:id, :sid, :start, current_timestamp, 'CRAWL_ERROR', :msg)
                """), {"id": p_id, "sid": session_id, "start": sessiooni_algus, "msg": str(e)[:200]})
                trans_err.execute(text("delete from queue where id = :id"), {"id": p_id})

except KeyboardInterrupt:
    print("\n🛑 Crawler peatatud kasutaja poolt.")

finally:
    with engine.begin() as trans_f:
        trans_f.execute(text("""
            update tai_load_sessions
            set
                dt_end = current_timestamp,
                status = :status
            where session_id = :sid
        """), {"sid": session_id, "status": final_status})

🚀 BLOKK 2: Korje algas. Sessioon: 20260218062845
✅ Andmebaas
✅ 01Rahvastik
✅ 02Haigestumus
✅ 03Tervishoiuteenused
✅ 04THressursid
✅ 05Uuringud
✅ 06Ravimistatistika
✅ 07Terviseprofiilid
✅ 08TerviseInfosysteem
✅ 01Rahvastik.02Synnid
✅ 01Rahvastik.03Abordid
✅ 01Rahvastik.04Surmad
✅ 01Rahvastik.05Eluiga
✅ 01Rahvastik.06TEaastad
✅ 01Rahvastik.07Tervisekaotus
✅ 02Haigestumus.01Esmashaigestumus
✅ 02Haigestumus.02Nakkushaigused
✅ 02Haigestumus.03Tuberkuloos
✅ 02Haigestumus.04PahaloomulisedKasvajad
✅ 02Haigestumus.05Psyyhikahaired
✅ 02Haigestumus.06Narkomaaniaravi
✅ 02Haigestumus.07Soeluuringud
✅ 02Haigestumus.08AMI
✅ 02Haigestumus.09Vigastused
✅ 03Tervishoiuteenused.01Vastuvotud
✅ 03Tervishoiuteenused.02Paevaravi
✅ 03Tervishoiuteenused.03Kiirabi
✅ 03Tervishoiuteenused.04Diagnostika
✅ 03Tervishoiuteenused.05Kirurgia
✅ 03Tervishoiuteenused.06Geriaatria
✅ 03Tervishoiuteenused.07KAV
✅ 04THressursid.01TTosutajad
✅ 04THressursid.02Ravivoodid
✅ 04THressursid.03Seadmed
✅ 04THressursid.04Verekomponendi

In [ ]:
# BLOKK 3.0: Schemade loomine käivita kui lood tühja skeemi muidu pole vaja käivitada
engine = sqlalchemy.create_engine(f'duckdb:///{db_path}')
with engine.connect() as conn:
    schemas = conn.execute(text("SELECT DISTINCT db_schema FROM tai_meta WHERE db_schema != ''")).fetchall()
    with engine.begin() as trans:
        for (sch,) in schemas:
            trans.execute(text(f"CREATE SCHEMA IF NOT EXISTS {sch}"))
print("✅ BLOKK 3.0: Schemad loodud")

In [12]:
# ############################################################################################
# BLOKK 3: INKREMENTAALNE LAADIMINE
# Valib uuenenud tabelid ja laeb need andmebaasi.
# Proovime parandada csvsid ja valida neile õige encodingu
# ############################################################################################

import os, requests, shutil

RAW_DIR, WORK_DIR = "/tmp/enc_test", f"/tmp/load_{session_id}"
os.makedirs(RAW_DIR, exist_ok=True); os.makedirs(WORK_DIR, exist_ok=True)

NULLS = "['..', '.', '-', ' ', '...', 'None', ' *']"
ENCODINGS = [None, 'windows-1257', 'windows-1252', 'utf-8']

#uue inkremendi kokku panemine
with engine.connect() as conn:
    candidates = conn.execute(text("""
        select
            m.id,
            m.src_url,
            m.table_name,
            m.db_schema,
            m.dt_updated_api
        from tai_meta m
        left join (
            select
                table_id,
                max(dt_updated_api) as last_api_upd
            from tai_load_log
            where result = 'SUCCESS'
            group by 1
        ) log on m.id = log.table_id
        where m.obj_type = 't'
          and (log.last_api_upd is null or m.dt_updated_api > log.last_api_upd)
    """)).fetchall()

print(f"🔄 Laadimine: {len(candidates)} tabelit.")

try:
    for i, (t_id, t_url, t_name, t_schema, t_api_upd) in enumerate(candidates, 1):
        raw_path = os.path.join(RAW_DIR, f"{t_name}.raw")
        work_path = os.path.join(WORK_DIR, f"{t_name}.csv")
        t_start = datetime.now()

        try:
            time.sleep(0.3)

            res = requests.post(t_url, json={"query": [], "response": {"format": "csv2"}}, timeout=60)
            if res.status_code != 200:
                continue

            with open(raw_path, 'wb') as f:
                f.write(res.content)

            success = False
            for enc in ENCODINGS:
                try:
                    if enc is None:
                        shutil.copy2(raw_path, work_path)
                    else:
                        with open(raw_path, 'rb') as rf:
                            decoded = rf.read().decode(enc, errors='strict')
                        with open(work_path, 'w', encoding='utf-8') as wf:
                            wf.write(decoded)

                    with engine.connect() as conn:
                        prompt = conn.execute(text(f"""
                            select prompt
                            from sniff_csv('{work_path}', strict_mode=false, nullstr={NULLS})
                        """)).scalar()

                        if prompt:
                            conn.execute(text(f"create schema if not exists {t_schema};"))
                            conn.execute(text(f"""
                                create or replace table "{t_schema}"."{t_name}"
                                as select * {prompt.strip().rstrip(";")};
                            """))

                            conn.execute(text("""
                                insert into tai_load_log (
                                    table_id,
                                    session_id,
                                    dt_start,
                                    dt_end,
                                    dt_updated_api,
                                    result,
                                    notes
                                ) values (
                                    :tid, :sid, :ds, current_timestamp, :api, 'SUCCESS', :enc
                                )
                            """), {
                                "tid": t_id, "sid": session_id, "ds": t_start,
                                "api": t_api_upd, "enc": enc or 'RAW'
                            })

                            print(f"✅ [{i}/{len(candidates)}] {t_name}")
                            success = True; break

                except KeyboardInterrupt:
                    break ## et saaks katkestada kui ei taha lõpuni oodata
                except:
                    continue

            if not success:
                with engine.begin() as trans_err:
                    trans_err.execute(text("""
                        insert into tai_load_log (
                            table_id,
                            session_id,
                            dt_start,
                            dt_end,
                            result,
                            notes
                        ) values (
                            :tid, :sid, :ds, current_timestamp, 'LOAD_ERROR', 'Fail'
                        )
                    """), {"tid": t_id, "sid": session_id, "ds": t_start})

        except KeyboardInterrupt:
            print("\n🛑 Laadimine katkestatud.")
            break
        except Exception:
            continue

except KeyboardInterrupt:
    pass

🔄 Laadimine: 2405 tabelit.
✅ [1/2405] tai_02Synnid_SR001_px
✅ [2/2405] tai_02Synnid_SR01_px
✅ [3/2405] tai_02Synnid_SR02_px
✅ [4/2405] tai_02Synnid_SR03_px
✅ [5/2405] tai_02Synnid_SR10_px
✅ [6/2405] tai_02Synnid_SR30_px
✅ [7/2405] tai_02Synnid_SR32_px
✅ [8/2405] tai_02Synnid_SR40_px
✅ [9/2405] tai_02Synnid_SR41_px
✅ [10/2405] tai_02Synnid_SR42_px
✅ [11/2405] tai_02Synnid_SR421_px
✅ [12/2405] tai_02Synnid_SR43_px
✅ [13/2405] tai_02Synnid_SR44_px
✅ [14/2405] tai_02Synnid_SR441_px
✅ [15/2405] tai_02Synnid_SR45_px
✅ [16/2405] tai_02Synnid_SR47_px
✅ [17/2405] tai_02Synnid_SR50_px
✅ [18/2405] tai_02Synnid_SR51_px
✅ [19/2405] tai_02Synnid_SR52_px
✅ [20/2405] tai_02Synnid_SR53_px
✅ [21/2405] tai_02Synnid_SR54_px
✅ [22/2405] tai_02Synnid_SR55_px
✅ [23/2405] tai_02Synnid_SR57_px
✅ [24/2405] tai_02Synnid_SR60_px
✅ [25/2405] tai_02Synnid_SR61_px
✅ [26/2405] tai_02Synnid_SR62_px
✅ [27/2405] tai_02Synnid_SR63_px
✅ [28/2405] tai_02Synnid_SR64_px
✅ [29/2405] tai_02Synnid_SR641_px
✅ [30/2405] tai_02Syn

In [ ]:
%%sql

select * from duckdb_tables();



In [19]:

!git add data/
!git commit -m "Esmased andmed 2026 veebruar"
!git push origin main

[main 515f6fa] Esmased andmed 2026 veebruar
 2406 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 data/tai_00Maakond_TKU100_px.parquet
 create mode 100644 data/tai_00Maakond_TKU101_px.parquet
 create mode 100644 data/tai_00Maakond_TKU102_px.parquet
 create mode 100644 data/tai_00Maakond_TKU103_px.parquet
 create mode 100644 data/tai_00Maakond_TKU104_px.parquet
 create mode 100644 data/tai_00Maakond_TKU105_px.parquet
 create mode 100644 data/tai_00Maakond_TKU106_px.parquet
 create mode 100644 data/tai_00Maakond_TKU107_px.parquet
 create mode 100644 data/tai_00Maakond_TKU108_px.parquet
 create mode 100644 data/tai_00Maakond_TKU109_px.parquet
 create mode 100644 data/tai_00Maakond_TKU110_px.parquet
 create mode 100644 data/tai_00Maakond_TKU111_px.parquet
 create mode 100644 data/tai_00Maakond_TKU112_px.parquet
 create mode 100644 data/tai_01Aastastatistika_RV20_px.parquet
 create mode 100644 data/tai_01Aastastatistika_RV21_px.parquet
 create mode 100644 data/tai_01Aasta

In [14]:
# ############################################################################################
# BLOKK 4: EKSPORT PARQUET FORMAATI
# Eksporteerib 't' tüüpi andmetabelid ja lisaks ka 'tai_meta' tabeli.
# Kasutab dünaamilist SQL-i ja dbt-stiilis vormistust.
# ############################################################################################

import os

# 1. Seadistame lokaalse kausta
local_export_path = '/content/tai_avaandmed/data/'
os.makedirs(local_export_path, exist_ok=True)

print(f"📂 Alustan eksporti: {local_export_path}")

try:
    # A. Eksporteerime andmetabelid ('t')
    with engine.connect() as conn:
        tables = conn.execute(text("""
            select
                db_schema,
                table_name
            from tai_meta
            where obj_type = 't'
              and table_name != ''
        """)).fetchall()

    if tables:
        for i, (schema, table) in enumerate(tables, 1):
            target_file = os.path.join(local_export_path, f"{table}.parquet")

            # Dünaamiline skeemi kontroll
            if not schema or schema.strip() == "":
                sql_query = f'select * from "{table}"'
            else:
                sql_query = f'select * from "{schema}"."{table}"'

            copy_sql = f"copy ({sql_query}) to '{target_file}' (format 'parquet', compression 'snappy')"

            try:
                with engine.begin() as trans:
                    trans.execute(text(copy_sql))
                if i % 20 == 0 or i == len(tables):
                    print(f"📦 Progress: {i}/{len(tables)} andmetabelit valmis.")
            except Exception as e:
                print(f"❌ Viga tabeliga {table}: {e}")
                continue

    # B. Eksporteerime tai_meta tabeli (sisukord)
    print("📋 Eksporteerin metaandmed (tai_meta)...")
    meta_file = os.path.join(local_export_path, "tai_meta.parquet")
    try:
        with engine.begin() as trans:
            trans.execute(text(f"copy (select * from tai_meta) to '{meta_file}' (format 'parquet', compression 'snappy')"))
        print("✅ tai_meta.parquet on lisatud.")
    except Exception as e:
        print(f"❌ Viga tai_meta eksportimisel: {e}")

    print(f"\n✨ Kõik failid on valmis kaustas: {local_export_path}")

except KeyboardInterrupt:
    print("\n🛑 Eksport katkestatud.")

📂 Alustan eksporti: /content/tai_avaandmed/data/
📦 Progress: 20/2405 andmetabelit valmis.
📦 Progress: 40/2405 andmetabelit valmis.
📦 Progress: 60/2405 andmetabelit valmis.
📦 Progress: 80/2405 andmetabelit valmis.
📦 Progress: 100/2405 andmetabelit valmis.
📦 Progress: 120/2405 andmetabelit valmis.
📦 Progress: 140/2405 andmetabelit valmis.
📦 Progress: 160/2405 andmetabelit valmis.
📦 Progress: 180/2405 andmetabelit valmis.
📦 Progress: 200/2405 andmetabelit valmis.
📦 Progress: 220/2405 andmetabelit valmis.
📦 Progress: 240/2405 andmetabelit valmis.
📦 Progress: 260/2405 andmetabelit valmis.
📦 Progress: 280/2405 andmetabelit valmis.
📦 Progress: 300/2405 andmetabelit valmis.
📦 Progress: 320/2405 andmetabelit valmis.
📦 Progress: 340/2405 andmetabelit valmis.
📦 Progress: 360/2405 andmetabelit valmis.
📦 Progress: 380/2405 andmetabelit valmis.
📦 Progress: 400/2405 andmetabelit valmis.
📦 Progress: 420/2405 andmetabelit valmis.
📦 Progress: 440/2405 andmetabelit valmis.
📦 Progress: 460/2405 andmetabel

In [ ]:
with engine.connect() as conn:
    # Kasutame duckdb_tables süsteemset vaadet
    query = text("""
        SELECT
            schema_name,
            count(*) as tabelite_arv,
            sum(estimated_size) as ridade_arv_hinnanguliselt
        FROM duckdb_tables()
        WHERE schema_name NOT IN ('information_schema', 'pg_catalog', 'main')
        GROUP BY schema_name
        ORDER BY tabelite_arv DESC;
    """)

    result = conn.execute(query).fetchall()

    print(f"{'SKEEM (Valdkond)':<30} | {'TABELEID':<10} | {'RIDU (u)':<10}")
    print("-" * 55)
    for row in result:
        # row[0] on schema_name, row[1] on count, row[2] on sum
        print(f"{str(row[0]):<30} | {str(row[1]):<10} | {str(row[2]):<10}")

SKEEM (Valdkond)               | TABELEID   | RIDU (u)  
-------------------------------------------------------
tai_05Uuringud                 | 1953       | 742385    
tai_04THressursid              | 140        | 1160061   
tai_01Rahvastik                | 96         | 1149084   
tai_02Haigestumus              | 95         | 1588135   
tai_03Tervishoiuteenused       | 80         | 1899897   
tai_07Terviseprofiilid         | 20         | 104334    
tai_06Ravimistatistika         | 16         | 10372     
tai_08TerviseInfosysteem       | 5          | 116013    
